# rhino3dm exercise 02

## import and util funcs

In [1]:
import rhino3dm
from dataclasses import dataclass, field
import random
import string

print('rhino3dm ver: ', rhino3dm.__version__)

def generate_id() -> str:
    return "".join(random.choices(string.ascii_lowercase, k=12))

rhino3dm ver:  8.6.0

## read `.3dm` file

In [2]:
rhino_f_path = "./data/nongke.3dm"
file3dm_obj = rhino3dm.File3dm.Read(rhino_f_path)
print('rhino3dm.File3dm object: ', file3dm_obj)

rhino3dm.File3dm object:  <rhino3dm._rhino3dm.File3dm object at 0x0000022099E25430>

## read `notes`

In [3]:
file3dm_notes = rhino3dm.File3dm.ReadNotes(rhino_f_path).split(';')[:-1]

for idx, note in enumerate(file3dm_notes):
    print(
        note.strip().split(':')[0].strip(),
        ' : ',
        note.strip().split(':')[1].strip()
    )

name  :  nongke
lati  :  20
long  :  100

> **File3dm.ReadNotes()**
>
> [***method***
> `rhino3dm.File3dm.ReadNotes()`](https://mcneel.github.io/rhino3dm/python/api/File3dm.html#rhino3dm.File3dm.ReadNotes)
>
> <br />
>
> Reads only the notes from an existing 3dm file.
>
> <br />
>
> Parameters:
>
> -   path (str) – The file from which to read the notes.
>
> Returns:
>
> -   The 3dm file notes.
>
> Return type:
>
> -   str

## read `Layer`

In [4]:
layer_table = file3dm_obj.Layers
print('layer table object: ', layer_table)
print('num of layers: ', len(layer_table))
print()

for idx, layer in enumerate(iter(layer_table)):
    print('layer ', idx, ' Object  : ', layer)
    print('      ', idx, ' Id      : ', layer.Id)
    print('      ', idx, ' Index   : ', layer.Index)
    print('      ', idx, ' Name    : ', layer.Name)
    print('      ', idx, ' Fullpath: ', layer.FullPath)
    print('      ', idx, ' Color   : ', layer.Color)
    print()

layer table object:  <rhino3dm._rhino3dm.File3dmLayerTable object at 0x0000022099DDBB70>
num of layers:  8

layer  0  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000022099DD9CB0>
       0  Id      :  212960d6-ad46-439d-8516-772ce85fde03
       0  Index   :  0
       0  Name    :  0
       0  Fullpath:  0
       0  Color   :  (0, 0, 0, 255)

layer  1  Object  :  <rhino3dm._rhino3dm.Layer object at 0x00000220980207F0>
       1  Id      :  9c23c957-4e0e-4574-8a15-78d5bc3a6a97
       1  Index   :  1
       1  Name    :  site_a
       1  Fullpath:  site_a
       1  Color   :  (0, 0, 0, 255)

layer  2  Object  :  <rhino3dm._rhino3dm.Layer object at 0x00000220F0360570>
       2  Id      :  25488de8-55a5-470b-a098-ea3c8c1220b3
       2  Index   :  2
       2  Name    :  cable
       2  Fullpath:  site_a::cable
       2  Color   :  (255, 191, 0, 255)

layer  3  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000022099DED070>
       3  Id      :  0428f404-44e4-47e7-ae60-72284d0a89f7
    

## read `ObjectsTable`

In [5]:
object_table = file3dm_obj.Objects
print('object table: ', object_table, '\n')

object table:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x0000022099DEC2F0> 


## create dataclass `OatPV`

In [6]:
@dataclass
class OatPV:
    idx: int
    outline: rhino3dm.PolylineCurve
    str_num: str = ''
    id: str = field(default_factory=generate_id)

## read `pv`

In [7]:
layer_idx_of_pv = 3
cnt_pv = 0
max_num_obj = 3
total_num_pv = 0

oatPV_list = []

for idx, obj in enumerate(iter(object_table)):
    # print the first 3 obj on a layer
    if (obj.Attributes.LayerIndex == layer_idx_of_pv) and (cnt_pv < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

            oatPV = OatPV(idx, obj.Geometry)
            oatPV_list.append(oatPV)

        cnt_pv += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pv):
        total_num_pv += 1

print('total num of pv: ', total_num_pv)
print()

for idx, oatPV in enumerate(oatPV_list):
    print('oatPV ', idx, ' : ', oatPV)

object  72  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2B3030>
        72  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x0000022099DCD270>
        72  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x0000022099DCD270>
        72  Attributes.Id         :  36f08c55-5986-4aa2-b0b8-ae56b26b043c
        72  Attributes.Name       :  
        72  Attributes.LayerIndex :  3
        72  Attributes.ObjectColor:  (0, 0, 0, 255)
        72  Attributes.Visible    :  True

            curve IsClosed     : True
            curve IsPlanar     : True
            curve IsPolyline   : True

            curve PointCount   : 5
            curve pt  0  :  35.10614556790845,209.9589626310085,0.0
            curve pt  1  :  35.28014104193071,210.94573999952675,0.0
            curve pt  2  :  36.95467106254545,210.65047517688606,0.0
            curve pt  3  :  36.78067558852319,209.6636978083678,0.0
            curve pt  4 

## read `cable`

In [8]:
layer_idx_of_cable = 1
cnt_cable = 0
max_num_obj = 3
total_num_cable = 0

for idx, obj in enumerate(iter(object_table)):
    # print the first 3 obj on a layer
    if (obj.Attributes.LayerIndex == layer_idx_of_cable) and (cnt_cable < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

        cnt_cable += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_cable):
        total_num_cable += 1

print('total num of cable: ', total_num_cable)

total num of cable:  0

## read `text`

In [9]:
layer_idx_of_text = 4
cnt_text = 0
max_num_obj = 3
total_num_text = 0

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text) and (cnt_text < max_num_obj):
        print('object ', idx, ' Object                  : ', obj)
        print('       ', idx, ' Geometry                : ', obj.Geometry)
        print('       ', idx, ' Attributes              : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
        print()

        # if geom is AnnotationBase, print its PlainText
        if isinstance(obj.Geometry, rhino3dm.AnnotationBase):
            print('            Geometry.PlainText     : ', obj.Geometry.PlainText)
            print()

        cnt_text += 1

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text):
        total_num_text += 1

print('total num of text: ', total_num_text)

object  2472  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A30F1F0>
        2472  Geometry                :  <rhino3dm._rhino3dm.AnnotationBase object at 0x000002209A2A4A70>
        2472  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000002209A2A4A70>
        2472  Attributes.Id           :  331127a8-6c28-4a2e-b56b-6d8c0879f793
        2472  Attributes.Name         :  
        2472  Attributes.LayerIndex   :  4
        2472  Attributes.ObjectColor  :  (0, 0, 0, 255)
        2472  Attributes.Visible      :  True

            Geometry.PlainText     :  5-5-9

object  2473  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2D71B0>
        2473  Geometry                :  <rhino3dm._rhino3dm.AnnotationBase object at 0x000002209A30F1F0>
        2473  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000002209A30F1F0>
        2473  Attributes.Id           :  889cae8f-6

## create dataclass `MyPt`

In [10]:
@dataclass
class OatPt:
    idx  : int
    point: rhino3dm.Point3d
    name : str

## read `pt`

In [11]:
layer_idx_of_pt = 2
cnt_pt = 0
max_num_obj = 3
total_num_pt = 0

oaPt_list_1 = []

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pt) and (cnt_pt < max_num_obj):
        print('object ', idx, ' Object                  : ', obj)
        print('       ', idx, ' Geometry                : ', obj.Geometry)
        print('       ', idx, ' Attributes              : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
        print()

        # if geom is Point, print its xyz
        if isinstance(obj.Geometry, rhino3dm.Point):
            pt = obj.Geometry.Location
            print('            Geometry.Location.X     : ', pt.X)
            print('            Geometry.Location.Y     : ', pt.Y)
            print('            Geometry.Location.Z     : ', pt.Z)

            oatPt = OatPt(idx, pt, str(idx))
            oaPt_list_1.append(oatPt)

            print()

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pt):
        total_num_pt += 1

print('total num of pt: ', total_num_pt)

object  0  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2D42F0>
        0  Geometry                :  <rhino3dm._rhino3dm.PolylineCurve object at 0x00000220F02E0AB0>
        0  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x00000220F02E0AB0>
        0  Attributes.Id           :  fa8bcce9-953e-47f0-bcfc-4de7a87e0c76
        0  Attributes.Name         :  
        0  Attributes.LayerIndex   :  2
        0  Attributes.ObjectColor  :  (0, 0, 0, 255)
        0  Attributes.Visible      :  True

object  1  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2B00B0>
        1  Geometry                :  <rhino3dm._rhino3dm.PolylineCurve object at 0x000002209A2D42F0>
        1  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000002209A2D42F0>
        1  Attributes.Id           :  a11caa23-6a3a-4dac-95b5-1c6005646eae
        1  Attributes.Name         :  
        1  Attrib

In [12]:
for idx, oatPt in enumerate(oaPt_list_1):
    print('myPt', idx, ' : ', oatPt)
    print('    type of idx in myPt:', type(oatPt.idx))
    print('    type of point in myPt:', type(oatPt.point))
    print('    type of name in myPt:', type(oatPt.name))

## test reading `TextDot` objects

In [13]:
rhino_f_path_text_dot = './data/test_text_dot.3dm'
file3dm_text_dot = rhino3dm.File3dm.Read(rhino_f_path_text_dot)
print(file3dm_text_dot)

In [14]:
text_dot_obj_table = file3dm_text_dot.Objects
print('textDot objects: ', text_dot_obj_table)

for idx, text_dot_obj in enumerate(iter(text_dot_obj_table)):
    print('textDot obj ', idx, ' : ', text_dot_obj)

    text_dot_geom = text_dot_obj.Geometry

    if isinstance(text_dot_geom, rhino3dm.TextDot):
        print('    textDot obj: ', text_dot_geom)
        print('    textDot point: ', text_dot_geom.Point)
        print('    textDot Text: ', text_dot_geom.Text)

    print()

textDot objects:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x00000220F02F6130>
textDot obj  0  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2B21B0>
    textDot obj:  <rhino3dm._rhino3dm.TextDot object at 0x000002209A2B1EB0>
    textDot point:  14.298569362095407,251.0734261769187,0.0
    textDot Text:  8-8-6

textDot obj  1  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002209A2B2CB0>
    textDot obj:  <rhino3dm._rhino3dm.TextDot object at 0x000002209A2B3AF0>
    textDot point:  7.675239846498812,248.7881056446808,0.0
    textDot Text:  8-8-5
